# Model 1: Basic Regression for Predicting MLB Scores

In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import sys
import datetime
import pickle as pkl
import matplotlib.pyplot as plt
#get parent directory:
import os
parentDirectory = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
sys.path.append('/Users/efriedlander/Dropbox/SportsBetting/mlb-predict')
from src.data import bbref_scrape

## Data Preparation

First import and parse the data scraped from baseball-reference. We will save the parsed data so we don't have to repeat that step every time we run the notebook.

In [2]:
scores = pkl.load(open(os.path.join(parentDirectory, 'data', 'boxscores2019.p'), 'rb'))
parsed_scores = bbref_scrape.parse_box_scores(scores)
pkl.dump(parsed_scores, open(os.path.join(parentDirectory, 'data', 'parsedData2019.p'), 'wb'))

In [3]:
out = pkl.load(open(os.path.join(parentDirectory, 'data', 'parsedData2019.p'), 'rb'))

The parser outputs four dataframes. For now we will only concern ourselves with the game and team level data.

In [4]:
game_level = out['Game']
team_level = out['Team']

For this first model we'll only consider the number of runs, hits, and starter for each game. Let's extract those, make sure they have the appropriate data types.

In [5]:
team_data = team_level[['GameID', 'Team', 'HomeAway', 'Runs', 'Hits', 'Starter']].astype({'GameID' : 'int64',
                                                                                                     'Team' : 'category',
                                                                                                     'HomeAway' : 'category',
                                                                                                     'Runs' : 'Int32',
                                                                                                     'Hits' : 'Int32',
                                                                                                     'Starter' : 'category'}, copy=False)

game_level['DateTime'] = pd.to_datetime(game_level['DateTime'], utc=True)
game_data = game_level[['GameID', 'DateTime']].astype({'GameID' : 'int64'})
team_data = team_data.merge(game_data, on='GameID')
team_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4858 entries, 0 to 4857
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype              
---  ------    --------------  -----              
 0   GameID    4858 non-null   int64              
 1   Team      4858 non-null   category           
 2   HomeAway  4858 non-null   category           
 3   Runs      4858 non-null   Int32              
 4   Hits      4858 non-null   Int32              
 5   Starter   4858 non-null   category           
 6   DateTime  4858 non-null   datetime64[ns, UTC]
dtypes: Int32(2), category(3), datetime64[ns, UTC](1), int64(1)
memory usage: 194.7 KB


We will find it convenient to number the games for each team.

In [6]:
team_data.sort_values(by='DateTime', inplace=True)
team_data['GameNum'] = team_data.groupby(by='Team')['DateTime'].rank('first', axis=0).astype('Int32')

Now compute the lagged running average for the number of runs and hits for each game.

In [7]:
team_data[['MeanRuns', 'MeanHits']] = team_data.groupby(by='Team').expanding().mean()[['Runs', 'Hits']].reset_index(level=0, drop=True)
team_data[['MeanRuns', 'MeanHits']] =team_data.groupby(by='Team')[['MeanRuns', 'MeanHits']].shift(1)

We will need the game odds. Due to formatting it will be easiest to add it to team_data first. Below we import the odds dataset and join the batting line and odds. Note that the odds dataset uses abbreviations so we'll need to define those and add them to the data set in order to join

In [8]:
team_abbrv = {'Atlanta Braves' : 'ATL', 
              'Arizona Diamondbacks' : 'ARI', 
              'Baltimore Orioles' : 'BAL', 
              'Boston Red Sox' : 'BOS', 
              'Chicago Cubs' : 'CUB', 
              'Chicago White Sox' : 'CWS', 
              'Cincinnati Reds' : 'CIN', 
              'Cleveland Indians' : 'CLE', 
              'Colorado Rockies' : 'COL', 
              'Detroit Tigers' : 'DET',
              'Kansas City Royals': 'KAN', 
              'Houston Astros' : 'HOU', 
              'Los Angeles Angels' : 'LAA', 
              'Los Angeles Dodgers' : 'LAD', 
              'Miami Marlins' : 'MIA', 
              'Florida Marlins' : 'FLA', 
              'Milwaukee Brewers' : 'MIL', 
              'Minnesota Twins' : 'MIN', 
              'New York Mets' : 'NYM', 
              'New York Yankees' : 'NYY', 
              'Oakland Athletics' : 'OAK',
              'Philadelphia Phillies' : 'PHI', 
              'Pittsburgh Pirates' : 'PIT', 
              'San Diego Padres' : 'SDG', 
              'Seattle Mariners' : 'SEA', 
              'San Francisco Giants' : 'SFO', 
              'St. Louis Cardinals' : 'STL', 
              'Tampa Bay Rays' : 'TAM', 
              'Texas Rangers' : 'TEX', 
              'Toronto Blue Jays' : 'TOR', 
              'Washington Nationals' : 'WAS'}
team_data['TeamAbr'] = team_data['Team'].apply(lambda x: team_abbrv[x])

odds = pd.read_csv(os.path.join(parentDirectory, 'data', 'mlbodds2019.csv'))
odds.rename(columns={'Team' : 'TeamAbr', 'Unnamed: 18' : 'Line Odds'}, inplace=True)
odds.sort_values(by='Date', inplace=True)
odds['GameNum']=odds.groupby('TeamAbr')['Date'].rank('first', axis=0).astype('Int32')

team_data = team_data.merge( odds[['TeamAbr', 'GameNum', 'Run Line', 'Line Odds']], on=['TeamAbr', 'GameNum'])

In [9]:
team_data[team_data['GameID']==-2984226396838710820]

,GameID,Team,HomeAway,Runs,Hits,Starter,DateTime,GameNum,MeanRuns,MeanHits,TeamAbr,Run Line,Line Odds


Now join the team data back to the game_level data in order to come up with our final dataset.

In [10]:
# team_data.drop(columns=['DateTime', 'HomeAway'], inplace=True)
model_data = game_level[['GameID', 'AwayTeam', 'HomeTeam', 'DateTime', 'AwayScore', 'HomeScore']]
model_data = model_data.merge(team_data, left_on=['GameID', 'AwayTeam'], right_on=['GameID', 'Team'])
model_data = model_data.merge(team_data, left_on=['GameID', 'HomeTeam'], right_on=['GameID', 'Team'], suffixes=('_away', '_home'))
model_data['ScoreDiff'] = model_data['AwayScore'] - model_data['HomeScore']
model_data.drop(columns=['Team_away', 'Team_home', 'Runs_away', 'Runs_home'], inplace=True)

In [11]:
model_data[model_data['GameID']==-2984226396838710820]

,GameID,AwayTeam,HomeTeam,DateTime_x,AwayScore,HomeScore,HomeAway_away,Hits_away,Starter_away,DateTime_y,...,Hits_home,Starter_home,DateTime,GameNum_home,MeanRuns_home,MeanHits_home,TeamAbr_home,Run Line_home,Line Odds_home,ScoreDiff


## Data Exploration

Now that we have all of the data we will use for our model let's explore it a little bit.

In [12]:
model_data[model_data['Run Line_home'] != -model_data['Run Line_away']][['Run Line_home', 'Run Line_away']]

,Run Line_home,Run Line_away
444,1.5,1.5
447,1.5,1.5
448,-1.5,-1.5
509,1.5,1.5
510,-1.5,-1.5
...,...,...
2326,1.5,1.5
2327,-1.5,-1.5
2352,1.5,1.5
2407,1.5,1.5


In [13]:
model_data.iloc[509, :]

GameID                  6986039953041558815
AwayTeam                     Houston Astros
HomeTeam                     Boston Red Sox
DateTime_x        2019-05-19 00:15:00+00:00
AwayScore                                 7
HomeScore                                 3
HomeAway_away                          Away
Hits_away                                13
Starter_away                  Corbin Martin
DateTime_y        2019-05-19 00:15:00+00:00
GameNum_away                             46
MeanRuns_away                       5.51111
MeanHits_away                       9.53333
TeamAbr_away                            HOU
Run Line_away                           1.5
Line Odds_away                         -120
HomeAway_home                          Home
Hits_home                                 5
Starter_home               Hector Velazquez
DateTime          2019-05-19 00:15:00+00:00
GameNum_home                             45
MeanRuns_home                          5.25
MeanHits_home                   

In [52]:
odds[(odds['TeamAbr']=='HOU')].iloc[:162]

,Date,Rot,VH,TeamAbr,Pitcher,1st,2nd,3rd,4th,5th,...,Final,Open,Close,Run Line,Line Odds,Open OU,Unnamed: 20,Close OU,Unnamed: 22,GameNum
22,328,923,V,HOU,JVERLANDER-R,0,0,3,1,1,...,5,-121,-128,-1.5,133,6.5,-110,7.0,100,1
44,329,961,V,HOU,GCOLE-R,0,0,2,0,0,...,2,-145,-138,-1.5,118,7.0,-110,7.0,-125,2
70,330,923,V,HOU,CMCHUGH-R,1,0,0,0,0,...,1,-130,-118,-1.5,141,7.5,-110,8.0,-125,3
96,331,969,V,HOU,WMILEY-L,0,0,1,0,0,...,1,-119,-113,-1.5,140,8.0,-110,8.5,-110,4
126,401,919,V,HOU,BPEACOCK-R,1,0,0,0,0,...,2,-165,-165,-1.5,-123,9.5,-110,9.5,-110,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4760,926,919,V,HOU,WMILEY-L,0,0,0,1,2,...,3,-245,-190,-1.5,-130,9.5,-110,9.0,-120,158
4786,927,975,V,HOU,JURQUIDY-R,0,1,0,0,0,...,4,-210,-210,-1.5,-140,10.0,-110,9.5,-105,159
4816,928,925,V,HOU,JVERLANDER-R,0,0,0,0,0,...,6,-450,-430,-1.5,-240,9.0,-110,9.0,-105,160
4844,929,971,V,HOU,GCOLE-R,1,2,2,3,0,...,8,-400,-410,-1.5,-230,8.5,-110,8.5,-125,161


In [53]:
team_data[(team_data['TeamAbr']=='HOU')].tail()

,GameID,Team,HomeAway,Runs,Hits,Starter,DateTime,GameNum,MeanRuns,MeanHits,TeamAbr,Run Line,Line Odds
4742,415175438487400985,Houston Astros,Away,3,7,Zack Greinke,2019-09-26 03:10:00+00:00,158,5.707006,9.528662,HOU,-1.5,-130
4765,147282896235485910,Houston Astros,Away,3,12,Wade Miley,2019-09-27 03:07:00+00:00,159,5.689873,9.512658,HOU,-1.5,-140
4790,-5919443220995324401,Houston Astros,Away,4,4,Jose Urquidy,2019-09-28 03:07:00+00:00,160,5.672956,9.528302,HOU,-1.5,-240
4825,-2556413535472837641,Houston Astros,Away,6,6,Justin Verlander,2019-09-29 02:07:00+00:00,161,5.662500,9.493750,HOU,-1.5,-230
4839,-8923962237882211415,Houston Astros,Away,8,13,Gerrit Cole,2019-09-29 20:07:00+00:00,162,5.664596,9.472050,HOU,-1.5,-120


In [35]:
game_level[game_level['GameID']==1096196326745839632]

,GameID,AwayTeam,HomeTeam,DateTime,Attendance,Venue,Duration,Details,AwayScore,HomeScore


In [17]:
odds.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4928 entries, 0 to 4927
Data columns (total 24 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Date         4928 non-null   int64  
 1   Rot          4928 non-null   int64  
 2   VH           4928 non-null   object 
 3   TeamAbr      4928 non-null   object 
 4   Pitcher      4928 non-null   object 
 5   1st          4928 non-null   int64  
 6   2nd          4928 non-null   int64  
 7   3rd          4928 non-null   int64  
 8   4th          4928 non-null   int64  
 9   5th          4928 non-null   int64  
 10  6th          4928 non-null   object 
 11  7th          4928 non-null   object 
 12  8th          4928 non-null   object 
 13  9th          4928 non-null   object 
 14  Final        4928 non-null   int64  
 15  Open         4928 non-null   object 
 16  Close        4928 non-null   int64  
 17  Run Line     4928 non-null   float64
 18  Line Odds    4928 non-null   int64  
 19  Open O